In [1]:
import numpy as np
import cv2
from mlutils import *
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

In [45]:
tf.config.list_physical_devices('GPU')

[]

# Generate Dataset and Store in images

In [2]:
window_duration=0.15
sampling_freq=2000
selected_muscles=[1,2,3]


window_length=int(window_duration*sampling_freq)

In [ ]:
subject_names=os.listdir(BASE_EXTRACTION_PATH)
print(subject_names)


for sub in subject_names:
    for f in range(1,10):
        t, ang, emg_s=read_extracted_combined_data(sub,f)
        emg_s=np.array(emg_s)
        class_sig=generate_classifier_signal(ang)

        mus_imgs=generate_classified_muscle_images(class_sig, emg_s, selected_muscles, window_duration=window_duration, num_examples=500)
        save_muscle_images(sub, f,mus_imgs)


['Abhoy', 'Alok', 'Tushar']














































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































# Load data from image files

In [3]:
import os
from sklearn.model_selection import train_test_split

subject_names=os.listdir(BASE_EXTRACTION_PATH)
num_data=500
emg_images=[]
emg_labels=[]

for sub in subject_names:
    for c in range(4):
        for i in range(1,10):
            for x in range(100):
                try:
                    img=cv2.imread(f'{BASE_EXTRACTION_PATH}/{sub}/Dataset/{i}/{c}/{x}.jpg', cv2.IMREAD_GRAYSCALE)/255.0
                    img=img.reshape((img.shape[0],img.shape[1], 1))
                    emg_images.append(img)
                    emg_labels.append([c])
                except TypeError:
                    print(f'Error taking {BASE_EXTRACTION_PATH}/{sub}/Dataset/{i}/{c}/{x}.jpg')


train_images, test_images, train_labels, test_labels=train_test_split(emg_images, emg_labels, train_size=0.80)

train_images=tf.convert_to_tensor(train_images)
test_images=tf.convert_to_tensor(test_images)
train_labels=tf.convert_to_tensor(train_labels)
test_labels=tf.convert_to_tensor(test_labels)

print(f'Training Size: {train_labels.shape}  Test Size: {test_labels.shape}')

Error taking D:/IIT_D/Sem2/BTP/emg_based_intension_prediction_elbow/Extracted_Data/Alok/Dataset/1/0/80.jpg
Error taking D:/IIT_D/Sem2/BTP/emg_based_intension_prediction_elbow/Extracted_Data/Alok/Dataset/1/0/81.jpg
Error taking D:/IIT_D/Sem2/BTP/emg_based_intension_prediction_elbow/Extracted_Data/Alok/Dataset/1/0/82.jpg
Error taking D:/IIT_D/Sem2/BTP/emg_based_intension_prediction_elbow/Extracted_Data/Alok/Dataset/1/0/83.jpg
Error taking D:/IIT_D/Sem2/BTP/emg_based_intension_prediction_elbow/Extracted_Data/Alok/Dataset/1/0/84.jpg
Error taking D:/IIT_D/Sem2/BTP/emg_based_intension_prediction_elbow/Extracted_Data/Alok/Dataset/1/0/85.jpg
Error taking D:/IIT_D/Sem2/BTP/emg_based_intension_prediction_elbow/Extracted_Data/Alok/Dataset/1/0/86.jpg
Error taking D:/IIT_D/Sem2/BTP/emg_based_intension_prediction_elbow/Extracted_Data/Alok/Dataset/1/0/87.jpg
Error taking D:/IIT_D/Sem2/BTP/emg_based_intension_prediction_elbow/Extracted_Data/Alok/Dataset/1/0/88.jpg
Error taking D:/IIT_D/Sem2/BTP/emg_ba

In [4]:
class CNNClassifier:
    def __init__(self, input_shape, save_file=None):
        self.model=models.Sequential()
        self.model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
        self.model.add(layers.MaxPooling2D((1,3)))
        self.model.add(layers.Conv2D(64, (1,10), activation='relu'))
        self.model.add(layers.AveragePooling2D((1,10)))
        self.model.add(layers.Conv2D(128, (1,5), activation='relu'))
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(64, activation='relu'))
        self.model.add(layers.Dense(4, activation='softmax'))
        self.model.summary()
        self.ckpt_path=save_file
        self.cb_ckpt=None
        if save_file!=None:
            self.cb_ckpt=callbacks.ModelCheckpoint(filepath=save_file, save_weights_only=True, verbose=1)

        self.model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

        self.history=None

    def train(self, train_data, train_label, epochs=10, test_data=None, test_label=None):
        cb=[self.cb_ckpt] if self.cb_ckpt!=None else None
        if test_data==None or test_label==None:
            self.history=self.model.fit(train_data,train_label, epochs=epochs)
        else:
            self.history=self.model.fit(train_data, train_label, epochs=epochs, validation_data=(test_data, test_label), callbacks=cb) 

        
    def predict(self, test_data):
        if self.cb_ckpt!=None:
            self.model.load_weights(self.ckpt_path)

        if len(test_data.shape)<=3:
            test_data=tf.convert_to_tensor([test_data])

        pred=self.model.predict(test_data)
        return pred



In [5]:
classifier=CNNClassifier((3, window_length, 1), './model_checkpoint.ckpt')
#classifier=CNNClassifier((3, window_length, 1))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 298, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 99, 32)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 90, 64)         20544     
                                                                 
 average_pooling2d (AverageP  (None, 1, 9, 64)         0         
 ooling2D)                                                       
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 5, 128)         41088     
                                                                 
 flatten (Flatten)           (None, 640)               0

In [129]:

classifier.train(train_images, train_labels, 40, test_images, test_labels)

Epoch 1/40
269/270 [============================>.] - ETA: 0s - loss: 1.1987 - accuracy: 0.4346
Epoch 1: saving model to .\model_checkpoint.ckpt
270/270 [==============================] - 3s 10ms/step - loss: 1.1988 - accuracy: 0.4341 - val_loss: 1.1227 - val_accuracy: 0.4652
Epoch 2/40
264/270 [============================>.] - ETA: 0s - loss: 1.0669 - accuracy: 0.5266
Epoch 2: saving model to .\model_checkpoint.ckpt
270/270 [==============================] - 2s 9ms/step - loss: 1.0676 - accuracy: 0.5267 - val_loss: 1.0398 - val_accuracy: 0.5371
Epoch 3/40
265/270 [============================>.] - ETA: 0s - loss: 0.9765 - accuracy: 0.5879
Epoch 3: saving model to .\model_checkpoint.ckpt
270/270 [==============================] - 3s 9ms/step - loss: 0.9768 - accuracy: 0.5864 - val_loss: 0.9602 - val_accuracy: 0.6053
Epoch 4/40
268/270 [============================>.] - ETA: 0s - loss: 0.9075 - accuracy: 0.6265
Epoch 4: saving model to .\model_checkpoint.ckpt
270/270 [=================

In [ ]:
import matplotlib.pyplot as plt
from signal_extraction import read_extracted_combined_data

%matplotlib qt
t,angle,emgs=read_extracted_combined_data('Abhoy',5)
emg_image=np.array(emgs[:3])

flex_ex=generate_classifier_signal(angle)
emg_image=emg_image.reshape((3,emgs[0].shape[0],1))
emg_image
out_sig=np.zeros(emg_image.shape[1])

for i in range(window_length, emg_image.shape[1], window_length):
    data=emg_image[:,i-window_length:i]
    if data.shape[1]==window_length:
        predicted_label=classifier.predict(data)
        print(predicted_label)
        out_sig[i:i+window_length]=np.argmax(predicted_label)



plt.plot(t,flex_ex)
plt.plot(t,out_sig)
plt.ylabel('Class Label')
plt.xlabel('Time')
plt.legend(['Actual', 'Predicted'])



1/1 [==============================] - 0s 14ms/step
[[9.9923933e-01 4.7311443e-07 7.6023478e-04 5.9032084e-14]]
1/1 [==============================] - 0s 14ms/step
[[9.9870431e-01 1.8028344e-05 1.2777308e-03 1.5230712e-12]]
1/1 [==============================] - 0s 13ms/step
[[9.9895227e-01 1.7070379e-05 1.0307125e-03 6.2699485e-12]]
1/1 [==============================] - 0s 15ms/step
[[9.9973482e-01 2.6858732e-05 2.3831821e-04 6.7786432e-10]]
1/1 [==============================] - 0s 17ms/step
[[9.9857938e-01 4.8131682e-04 9.3653961e-04 2.8596451e-06]]
1/1 [==============================] - 0s 15ms/step
[[4.583747e-02 9.541626e-01 7.323089e-13 8.228440e-16]]
1/1 [==============================] - 0s 15ms/step
[[9.9910754e-01 7.4975233e-04 1.4267550e-04 2.3281383e-09]]
1/1 [==============================] - 0s 14ms/step
[[8.93486917e-01 1.04493424e-01 2.01957510e-03 6.72072886e-08]]
1/1 [==============================] - 0s 15ms/step
[[0.14546482 0.8364299  0.01247957 0.00562578]]
1/1 

In [15]:
from sklearn.metrics import confusion_matrix

conf_pred_labels=np.argmax(classifier.predict(train_images), axis=1)
print(conf_pred_labels)
cm= confusion_matrix(conf_pred_labels, train_labels)

print(cm)

270/270 [==============================] - 1s 3ms/step
[2 0 3 ... 2 3 2]
[[1847   61  172   14]
 [  76 1839   88   58]
 [ 135   67 1696   46]
 [  76  200  209 2036]]


In [16]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()
